In [3]:
import torch
from torch.nn import LSTM
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.func import functional_call
from torch.nn.functional import normalize

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

from dataset import TSDataset
from models import LSTMModel, AttentionModel, AttentionPMModel
from train_utils import train_and_test
from neural_memory import NeuralMemory
from titans import MACTitanLayer, MACTitan

from tqdm import tqdm
from copy import deepcopy

torch.set_grad_enabled(True)

# ===========================================================================================

In [5]:
train_data = TSDataset('weather', 'train', False)
valid_data = TSDataset('weather', 'valid', False)
test_data = TSDataset('weather', 'test', False)

_train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
_valid_loader = DataLoader(valid_data, batch_size=4, shuffle=True)
_test_loader = DataLoader(test_data, batch_size=4, shuffle=True)

In [4]:
model = LSTMModel(5, 16, 16)

optimizer = optim.Adam(model.parameters())

train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 50)

Training:


100%|██████████| 50/50 [00:37<00:00,  1.32it/s]


 Testing the best model:
Test MSE: 0.035601225142416204




/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(


LSTMModel(
  (emb_layer): Linear(in_features=5, out_features=16, bias=True)
  (relu): ReLU()
  (lstm): LSTM(16, 16, batch_first=True)
  (final_layer): Linear(in_features=16, out_features=1, bias=True)
)

In [5]:
model = AttentionModel(5, 16, 16)

optimizer = optim.Adam(model.parameters())

train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 50)

Training:


  0%|          | 0/50 [00:00<?, ?it/s]/content/models.py:64: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  indices = torch.from_numpy(sliding_window_view(np.arange(total_len), self.seq_len))
100%|██████████| 50/50 [09:42<00:00, 11.64s/it]



 Testing the best model:
Test MSE: 0.03525110926283033



AttentionModel(
  (emb_layer): Linear(in_features=5, out_features=16, bias=True)
  (att_layers): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
      )
      (linear1): Linear(in_features=16, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=16, bias=True)
      (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (activation): SiLU()
    )
  )
  (final_layer): Linear(in_features=256, out_features=1, bias=True)
)

In [6]:
model = AttentionPMModel(5, 16, 16, 4)

optimizer = optim.Adam(model.parameters())

train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 50)

Training:


100%|██████████| 50/50 [11:45<00:00, 14.11s/it]



 Testing the best model:
Test MSE: 0.03569375570667417



AttentionPMModel(
  (emb_layer): Linear(in_features=5, out_features=16, bias=True)
  (att_layers): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
      )
      (linear1): Linear(in_features=16, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=16, bias=True)
      (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (activation): SiLU()
    )
  )
  (final_layer): Linear(in_features=320, out_features=1, bias=True)
)

In [6]:
model = MACTitan(
    input_dim = 5,
    hidden_dim = 16,
    output_dim = 1,
    context_window = 16,
    pm_len = 4,
    n_layers = 2,
    n_layers_nmm = 2,
    eta=0.9,
    theta=0.01
).cuda()

optimizer = optim.Adam(model.outer_params)

best_model = train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 10, True)

Training:


  0%|          | 0/10 [00:00<?, ?it/s]/content/titans.py:155: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  stz = torch.from_numpy(x[:,:-residual].reshape(x.shape[0], -1, self.context_window, self.context_window, self.input_dim)).cuda()
 10%|█         | 1/10 [01:35<14:18, 95.44s/it]

Validation loss in Epoch 0: 0.03467170451032488


 20%|██        | 2/10 [03:10<12:40, 95.01s/it]

Validation loss in Epoch 1: 0.03373166181539235


...

 90%|█████████ | 9/10 [14:11<01:34, 94.10s/it]

Validation loss in Epoch 8: 0.036109270429924914


100%|██████████| 10/10 [15:45<00:00, 94.57s/it]

Validation loss in Epoch 9: 0.03277809196396878

 Testing the best model:


Test MSE: 0.03569832993181128



In [7]:
model = MACTitan(
    input_dim = 5,
    hidden_dim = 16,
    output_dim = 1,
    context_window = 16,
    pm_len = 4,
    n_layers = 2,
    n_layers_nmm = 1,
    eta=0.9,
    theta=0.01
).cuda()

optimizer = optim.Adam(model.outer_params)

best_model = train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 10, True)

Training:


 10%|█         | 1/10 [01:24<12:41, 84.57s/it]

Validation loss in Epoch 0: 0.031554996830068134


 20%|██        | 2/10 [02:50<11:24, 85.60s/it]

Validation loss in Epoch 1: 0.02973860088539751


...

 90%|█████████ | 9/10 [12:49<01:25, 85.22s/it]

Validation loss in Epoch 8: 0.03377002207072158


100%|██████████| 10/10 [14:15<00:00, 85.50s/it]

Validation loss in Epoch 9: 0.03529415703133533

 Testing the best model:


Test MSE: 0.037331398498070864

